# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [5]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))


# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 1s 504us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 136us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 103us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 151us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 140us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 126us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [

1051/1051 [==============================] - 0s 107us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 99us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 96us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 106us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 96us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 95us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 105us/step - loss: n

1051/1051 [==============================] - 0s 105us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 106/150
1051/1051 [==============================] - 0s 104us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 110us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 107us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 103us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 103us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 97us/step 

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [6]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [7]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [8]:
# Train the model 
normalized_input_model.fit(X_train, 
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 289us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 2/150
1051/1051 [==============================] - 0s 143us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 3/150
1051/1051 [==============================] - 0s 149us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 4/150
1051/1051 [==============================] - 0s 234us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 5/150
1051/1051 [==============================] - 0s 142us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 6/150
1051/1051 [==============================] - 0s 101us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 7/150
1051/1051 [

1051/1051 [==============================] - 0s 95us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 54/150
1051/1051 [==============================] - 0s 100us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 55/150
1051/1051 [==============================] - 0s 136us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 56/150
1051/1051 [==============================] - 0s 229us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 57/150
1051/1051 [==============================] - 0s 148us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 58/150
1051/1051 [==============================] - 0s 155us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 59/150
1051/1051 [==============================] - 0s 165us/step - loss

1051/1051 [==============================] - 0s 104us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 106/150
1051/1051 [==============================] - 0s 107us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 107/150
1051/1051 [==============================] - 0s 99us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 108/150
1051/1051 [==============================] - 0s 103us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 109/150
1051/1051 [==============================] - 0s 156us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 110/150
1051/1051 [==============================] - 0s 196us/step - loss: nan - mean_squared_error: nan - val_loss: nan - val_mean_squared_error: nan
Epoch 111/150
1051/1051 [==============================] - 0s 105us/step 

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [9]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [10]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 1s 511us/step - loss: 0.4368 - mean_squared_error: 0.4368 - val_loss: 0.1992 - val_mean_squared_error: 0.1992
Epoch 2/150
1051/1051 [==============================] - 0s 179us/step - loss: 0.2274 - mean_squared_error: 0.2274 - val_loss: 0.1613 - val_mean_squared_error: 0.1613
Epoch 3/150
1051/1051 [==============================] - 0s 283us/step - loss: 0.1842 - mean_squared_error: 0.1842 - val_loss: 0.1589 - val_mean_squared_error: 0.1589
Epoch 4/150
1051/1051 [==============================] - 0s 165us/step - loss: 0.1643 - mean_squared_error: 0.1643 - val_loss: 0.1375 - val_mean_squared_error: 0.1375
Epoch 5/150
1051/1051 [==============================] - 0s 182us/step - loss: 0.1480 - mean_squared_error: 0.1480 - val_loss: 0.1320 - val_mean_squared_error: 0.1320
Epoch 6/150
1051/1051 [==============================] - 0s 181us/step - loss: 0.1330 - mean_squared_error: 0.1330 - v

1051/1051 [==============================] - 0s 98us/step - loss: 0.0327 - mean_squared_error: 0.0327 - val_loss: 0.1031 - val_mean_squared_error: 0.1031
Epoch 50/150
1051/1051 [==============================] - 0s 104us/step - loss: 0.0316 - mean_squared_error: 0.0316 - val_loss: 0.1014 - val_mean_squared_error: 0.1014
Epoch 51/150
1051/1051 [==============================] - 0s 118us/step - loss: 0.0315 - mean_squared_error: 0.0315 - val_loss: 0.1026 - val_mean_squared_error: 0.1026
Epoch 52/150
1051/1051 [==============================] - 0s 210us/step - loss: 0.0308 - mean_squared_error: 0.0308 - val_loss: 0.1029 - val_mean_squared_error: 0.1029
Epoch 53/150
1051/1051 [==============================] - 0s 107us/step - loss: 0.0300 - mean_squared_error: 0.0300 - val_loss: 0.1030 - val_mean_squared_error: 0.1030
Epoch 54/150
1051/1051 [==============================] - 0s 103us/step - loss: 0.0291 - mean_squared_error: 0.0291 - val_loss: 0.1043 - val_mean_squared_error: 0.1043
Epoch 

Epoch 98/150
1051/1051 [==============================] - 0s 180us/step - loss: 0.0145 - mean_squared_error: 0.0145 - val_loss: 0.1102 - val_mean_squared_error: 0.1102
Epoch 99/150
1051/1051 [==============================] - 0s 172us/step - loss: 0.0143 - mean_squared_error: 0.0143 - val_loss: 0.1118 - val_mean_squared_error: 0.1118
Epoch 100/150
1051/1051 [==============================] - 0s 155us/step - loss: 0.0141 - mean_squared_error: 0.0141 - val_loss: 0.1109 - val_mean_squared_error: 0.1109
Epoch 101/150
1051/1051 [==============================] - 0s 205us/step - loss: 0.0141 - mean_squared_error: 0.0141 - val_loss: 0.1122 - val_mean_squared_error: 0.1122
Epoch 102/150
1051/1051 [==============================] - 0s 162us/step - loss: 0.0137 - mean_squared_error: 0.0137 - val_loss: 0.1116 - val_mean_squared_error: 0.1116
Epoch 103/150
1051/1051 [==============================] - 0s 144us/step - loss: 0.0136 - mean_squared_error: 0.0136 - val_loss: 0.1120 - val_mean_squared_er

1051/1051 [==============================] - 0s 131us/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.1163 - val_mean_squared_error: 0.1163
Epoch 147/150
1051/1051 [==============================] - 0s 103us/step - loss: 0.0084 - mean_squared_error: 0.0084 - val_loss: 0.1169 - val_mean_squared_error: 0.1169
Epoch 148/150
1051/1051 [==============================] - 0s 115us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1169 - val_mean_squared_error: 0.1169
Epoch 149/150
1051/1051 [==============================] - 0s 127us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1167 - val_mean_squared_error: 0.1167
Epoch 150/150
1051/1051 [==============================] - 0s 165us/step - loss: 0.0082 - mean_squared_error: 0.0082 - val_loss: 0.1179 - val_mean_squared_error: 0.1179


Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [11]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 97us/step


[0.007790016924889739, 0.007790016924889739]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [12]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 80us/step


[0.11790674086306031, 0.11790674086306031]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [13]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

26983.657126742342

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [14]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,), kernel_initializer='he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 347us/step - loss: 0.5083 - mean_squared_error: 0.5083 - val_loss: 0.1699 - val_mean_squared_error: 0.1699
Epoch 2/150
1051/1051 [==============================] - 0s 121us/step - loss: 0.2218 - mean_squared_error: 0.2218 - val_loss: 0.1432 - val_mean_squared_error: 0.1432
Epoch 3/150
1051/1051 [==============================] - 0s 112us/step - loss: 0.1798 - mean_squared_error: 0.1798 - val_loss: 0.1379 - val_mean_squared_error: 0.1379
Epoch 4/150
1051/1051 [==============================] - 0s 148us/step - loss: 0.1568 - mean_squared_error: 0.1568 - val_loss: 0.1251 - val_mean_squared_error: 0.1251
Epoch 5/150
1051/1051 [==============================] - 0s 112us/step - loss: 0.1410 - mean_squared_error: 0.1410 - val_loss: 0.1196 - val_mean_squared_error: 0.1196
Epoch 6/150
1051/1051 [==============================] - 0s 139us/step - loss: 0.1266 - mean_squared_error: 0.1266 - v

1051/1051 [==============================] - 0s 96us/step - loss: 0.0314 - mean_squared_error: 0.0314 - val_loss: 0.1104 - val_mean_squared_error: 0.1104
Epoch 50/150
1051/1051 [==============================] - 0s 91us/step - loss: 0.0303 - mean_squared_error: 0.0303 - val_loss: 0.1087 - val_mean_squared_error: 0.1087
Epoch 51/150
1051/1051 [==============================] - 0s 93us/step - loss: 0.0296 - mean_squared_error: 0.0296 - val_loss: 0.1089 - val_mean_squared_error: 0.1089
Epoch 52/150
1051/1051 [==============================] - 0s 93us/step - loss: 0.0293 - mean_squared_error: 0.0293 - val_loss: 0.1099 - val_mean_squared_error: 0.1099
Epoch 53/150
1051/1051 [==============================] - 0s 92us/step - loss: 0.0287 - mean_squared_error: 0.0287 - val_loss: 0.1100 - val_mean_squared_error: 0.1100
Epoch 54/150
1051/1051 [==============================] - 0s 99us/step - loss: 0.0281 - mean_squared_error: 0.0281 - val_loss: 0.1101 - val_mean_squared_error: 0.1101
Epoch 55/15

1051/1051 [==============================] - 0s 145us/step - loss: 0.0146 - mean_squared_error: 0.0146 - val_loss: 0.1173 - val_mean_squared_error: 0.1173
Epoch 99/150
1051/1051 [==============================] - 0s 159us/step - loss: 0.0144 - mean_squared_error: 0.0144 - val_loss: 0.1186 - val_mean_squared_error: 0.1186
Epoch 100/150
1051/1051 [==============================] - 0s 132us/step - loss: 0.0142 - mean_squared_error: 0.0142 - val_loss: 0.1181 - val_mean_squared_error: 0.1181
Epoch 101/150
1051/1051 [==============================] - 0s 143us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1181 - val_mean_squared_error: 0.1181
Epoch 102/150
1051/1051 [==============================] - 0s 102us/step - loss: 0.0140 - mean_squared_error: 0.0140 - val_loss: 0.1175 - val_mean_squared_error: 0.1175
Epoch 103/150
1051/1051 [==============================] - 0s 131us/step - loss: 0.0138 - mean_squared_error: 0.0138 - val_loss: 0.1176 - val_mean_squared_error: 0.1176
E

Epoch 147/150
1051/1051 [==============================] - 0s 120us/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.1232 - val_mean_squared_error: 0.1232
Epoch 148/150
1051/1051 [==============================] - 0s 107us/step - loss: 0.0086 - mean_squared_error: 0.0086 - val_loss: 0.1238 - val_mean_squared_error: 0.1238
Epoch 149/150
1051/1051 [==============================] - 0s 158us/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.1234 - val_mean_squared_error: 0.1234
Epoch 150/150
1051/1051 [==============================] - 0s 140us/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.1242 - val_mean_squared_error: 0.1242


Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [16]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 70us/step


[0.00826145356811661, 0.00826145356811661]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [17]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 279us/step


[0.12416238135201396, 0.12416238135201396]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [18]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, activation='relu', kernel_initializer='lecun_normal', input_shape=(n_features,)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 364us/step - loss: 0.4769 - mean_squared_error: 0.4769 - val_loss: 0.1464 - val_mean_squared_error: 0.1464
Epoch 2/150
1051/1051 [==============================] - 0s 111us/step - loss: 0.2096 - mean_squared_error: 0.2096 - val_loss: 0.1231 - val_mean_squared_error: 0.1231
Epoch 3/150
1051/1051 [==============================] - 0s 124us/step - loss: 0.1698 - mean_squared_error: 0.1698 - val_loss: 0.1167 - val_mean_squared_error: 0.1167
Epoch 4/150
1051/1051 [==============================] - 0s 157us/step - loss: 0.1479 - mean_squared_error: 0.1479 - val_loss: 0.1085 - val_mean_squared_error: 0.1085
Epoch 5/150
1051/1051 [==============================] - 0s 135us/step - loss: 0.1331 - mean_squared_error: 0.1331 - val_loss: 0.1062 - val_mean_squared_error: 0.1062
Epoch 6/150
1051/1051 [==============================] - 0s 108us/step - loss: 0.1196 - mean_squared_error: 0.1196 - v

1051/1051 [==============================] - 0s 104us/step - loss: 0.0276 - mean_squared_error: 0.0276 - val_loss: 0.1196 - val_mean_squared_error: 0.1196
Epoch 50/150
1051/1051 [==============================] - 0s 104us/step - loss: 0.0267 - mean_squared_error: 0.0267 - val_loss: 0.1163 - val_mean_squared_error: 0.1163
Epoch 51/150
1051/1051 [==============================] - 0s 101us/step - loss: 0.0261 - mean_squared_error: 0.0261 - val_loss: 0.1176 - val_mean_squared_error: 0.1176
Epoch 52/150
1051/1051 [==============================] - 0s 107us/step - loss: 0.0259 - mean_squared_error: 0.0259 - val_loss: 0.1190 - val_mean_squared_error: 0.1190
Epoch 53/150
1051/1051 [==============================] - 0s 107us/step - loss: 0.0253 - mean_squared_error: 0.0253 - val_loss: 0.1195 - val_mean_squared_error: 0.1195
Epoch 54/150
1051/1051 [==============================] - 0s 115us/step - loss: 0.0247 - mean_squared_error: 0.0247 - val_loss: 0.1194 - val_mean_squared_error: 0.1194
Epoch

Epoch 98/150
1051/1051 [==============================] - 0s 117us/step - loss: 0.0128 - mean_squared_error: 0.0128 - val_loss: 0.1281 - val_mean_squared_error: 0.1281
Epoch 99/150
1051/1051 [==============================] - 0s 106us/step - loss: 0.0126 - mean_squared_error: 0.0126 - val_loss: 0.1305 - val_mean_squared_error: 0.1305
Epoch 100/150
1051/1051 [==============================] - 0s 109us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1302 - val_mean_squared_error: 0.1302
Epoch 101/150
1051/1051 [==============================] - 0s 250us/step - loss: 0.0124 - mean_squared_error: 0.0124 - val_loss: 0.1308 - val_mean_squared_error: 0.1308
Epoch 102/150
1051/1051 [==============================] - 0s 122us/step - loss: 0.0122 - mean_squared_error: 0.0122 - val_loss: 0.1290 - val_mean_squared_error: 0.1290
Epoch 103/150
1051/1051 [==============================] - 0s 99us/step - loss: 0.0120 - mean_squared_error: 0.0120 - val_loss: 0.1298 - val_mean_squared_err

1051/1051 [==============================] - 0s 148us/step - loss: 0.0076 - mean_squared_error: 0.0076 - val_loss: 0.1356 - val_mean_squared_error: 0.1356
Epoch 147/150
1051/1051 [==============================] - 0s 228us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.1360 - val_mean_squared_error: 0.1360
Epoch 148/150
1051/1051 [==============================] - 0s 112us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.1372 - val_mean_squared_error: 0.1372
Epoch 149/150
1051/1051 [==============================] - 0s 109us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.1353 - val_mean_squared_error: 0.1353
Epoch 150/150
1051/1051 [==============================] - 0s 114us/step - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.1367 - val_mean_squared_error: 0.1367


Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 86us/step


[0.00701443953342119, 0.00701443953342119]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [20]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 117us/step


[0.13666676892810448, 0.13666676892810448]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [21]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 449us/step - loss: 0.3201 - mean_squared_error: 0.3201 - val_loss: 0.1366 - val_mean_squared_error: 0.1366
Epoch 2/150
1051/1051 [==============================] - 0s 145us/step - loss: 0.1677 - mean_squared_error: 0.1677 - val_loss: 0.1112 - val_mean_squared_error: 0.1112
Epoch 3/150
1051/1051 [==============================] - 0s 123us/step - loss: 0.1249 - mean_squared_error: 0.1249 - val_loss: 0.1278 - val_mean_squared_error: 0.1278
Epoch 4/150
1051/1051 [==============================] - 0s 132us/step - loss: 0.0992 - mean_squared_error: 0.0992 - val_loss: 0.1041 - val_mean_squared_error: 0.1041
Epoch 5/150
1051/1051 [==============================] - 0s 126us/step - loss: 0.0888 - mean_squared_error: 0.0888 - val_loss: 0.0995 - val_mean_squared_error: 0.0995
Epoch 6/150
1051/1051 [==============================] - 0s 136us/step - loss: 0.0668 - mean_squared_error: 0.0668 - v

1051/1051 [==============================] - 0s 128us/step - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.0897 - val_mean_squared_error: 0.0897
Epoch 50/150
1051/1051 [==============================] - 0s 109us/step - loss: 0.0091 - mean_squared_error: 0.0091 - val_loss: 0.1130 - val_mean_squared_error: 0.1130
Epoch 51/150
1051/1051 [==============================] - 0s 113us/step - loss: 0.0067 - mean_squared_error: 0.0067 - val_loss: 0.1168 - val_mean_squared_error: 0.1168
Epoch 52/150
1051/1051 [==============================] - 0s 131us/step - loss: 0.0091 - mean_squared_error: 0.0091 - val_loss: 0.0950 - val_mean_squared_error: 0.0950
Epoch 53/150
1051/1051 [==============================] - 0s 103us/step - loss: 0.0095 - mean_squared_error: 0.0095 - val_loss: 0.0894 - val_mean_squared_error: 0.0894
Epoch 54/150
1051/1051 [==============================] - 0s 108us/step - loss: 0.0073 - mean_squared_error: 0.0073 - val_loss: 0.0922 - val_mean_squared_error: 0.0922
Epoch

Epoch 98/150
1051/1051 [==============================] - 0s 172us/step - loss: 0.0051 - mean_squared_error: 0.0051 - val_loss: 0.0778 - val_mean_squared_error: 0.0778
Epoch 99/150
1051/1051 [==============================] - 0s 236us/step - loss: 0.0045 - mean_squared_error: 0.0045 - val_loss: 0.0855 - val_mean_squared_error: 0.0855
Epoch 100/150
1051/1051 [==============================] - 0s 117us/step - loss: 0.0058 - mean_squared_error: 0.0058 - val_loss: 0.0847 - val_mean_squared_error: 0.0847
Epoch 101/150
1051/1051 [==============================] - 0s 105us/step - loss: 0.0040 - mean_squared_error: 0.0040 - val_loss: 0.0815 - val_mean_squared_error: 0.0815
Epoch 102/150
1051/1051 [==============================] - 0s 114us/step - loss: 0.0046 - mean_squared_error: 0.0046 - val_loss: 0.0935 - val_mean_squared_error: 0.0935
Epoch 103/150
1051/1051 [==============================] - 0s 105us/step - loss: 0.0052 - mean_squared_error: 0.0052 - val_loss: 0.0878 - val_mean_squared_er

1051/1051 [==============================] - 0s 135us/step - loss: 0.0045 - mean_squared_error: 0.0045 - val_loss: 0.0797 - val_mean_squared_error: 0.0797
Epoch 147/150
1051/1051 [==============================] - 0s 126us/step - loss: 0.0032 - mean_squared_error: 0.0032 - val_loss: 0.0971 - val_mean_squared_error: 0.0971
Epoch 148/150
1051/1051 [==============================] - 0s 123us/step - loss: 0.0040 - mean_squared_error: 0.0040 - val_loss: 0.0870 - val_mean_squared_error: 0.0870
Epoch 149/150
1051/1051 [==============================] - 0s 148us/step - loss: 0.0044 - mean_squared_error: 0.0044 - val_loss: 0.0875 - val_mean_squared_error: 0.0875
Epoch 150/150
1051/1051 [==============================] - 0s 131us/step - loss: 0.0026 - mean_squared_error: 0.0026 - val_loss: 0.0884 - val_mean_squared_error: 0.0884


Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 73us/step


[0.0019590235326339897, 0.0019590235326339897]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 166us/step


[0.08837840840673719, 0.08837840840673719]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [26]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])
# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 1s 533us/step - loss: 0.3803 - mean_squared_error: 0.3803 - val_loss: 0.1649 - val_mean_squared_error: 0.1649
Epoch 2/150
1051/1051 [==============================] - 0s 136us/step - loss: 0.1611 - mean_squared_error: 0.1611 - val_loss: 0.1332 - val_mean_squared_error: 0.1332
Epoch 3/150
1051/1051 [==============================] - 0s 125us/step - loss: 0.1127 - mean_squared_error: 0.1127 - val_loss: 0.1041 - val_mean_squared_error: 0.1041
Epoch 4/150
1051/1051 [==============================] - 0s 141us/step - loss: 0.0870 - mean_squared_error: 0.0870 - val_loss: 0.1022 - val_mean_squared_error: 0.1022
Epoch 5/150
1051/1051 [==============================] - 0s 133us/step - loss: 0.0739 - mean_squared_error: 0.0739 - val_loss: 0.1006 - val_mean_squared_error: 0.1006
Epoch 6/150
1051/1051 [==============================] - 0s 127us/step - loss: 0.0565 - mean_squared_error: 0.0565 - v

1051/1051 [==============================] - 0s 136us/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.1007 - val_mean_squared_error: 0.1007
Epoch 50/150
1051/1051 [==============================] - 0s 132us/step - loss: 0.0012 - mean_squared_error: 0.0012 - val_loss: 0.0951 - val_mean_squared_error: 0.0951
Epoch 51/150
1051/1051 [==============================] - 0s 132us/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.1009 - val_mean_squared_error: 0.1009
Epoch 52/150
1051/1051 [==============================] - 0s 133us/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0939 - val_mean_squared_error: 0.0939
Epoch 53/150
1051/1051 [==============================] - 0s 137us/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0986 - val_mean_squared_error: 0.0986
Epoch 54/150
1051/1051 [==============================] - 0s 129us/step - loss: 0.0012 - mean_squared_error: 0.0012 - val_loss: 0.0949 - val_mean_squared_error: 0.0949
Epoch

Epoch 98/150
1051/1051 [==============================] - 0s 122us/step - loss: 0.0096 - mean_squared_error: 0.0096 - val_loss: 0.0959 - val_mean_squared_error: 0.0959
Epoch 99/150
1051/1051 [==============================] - 0s 146us/step - loss: 0.0047 - mean_squared_error: 0.0047 - val_loss: 0.0859 - val_mean_squared_error: 0.0859
Epoch 100/150
1051/1051 [==============================] - 0s 155us/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0896 - val_mean_squared_error: 0.0896
Epoch 101/150
1051/1051 [==============================] - 0s 153us/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0875 - val_mean_squared_error: 0.0875
Epoch 102/150
1051/1051 [==============================] - 0s 135us/step - loss: 0.0013 - mean_squared_error: 0.0013 - val_loss: 0.0914 - val_mean_squared_error: 0.0914
Epoch 103/150
1051/1051 [==============================] - 0s 123us/step - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0884 - val_mean_squared_er

Epoch 146/150
1051/1051 [==============================] - 0s 132us/step - loss: 0.0034 - mean_squared_error: 0.0034 - val_loss: 0.0890 - val_mean_squared_error: 0.0890
Epoch 147/150
1051/1051 [==============================] - 0s 141us/step - loss: 0.0020 - mean_squared_error: 0.0020 - val_loss: 0.0883 - val_mean_squared_error: 0.0883
Epoch 148/150
1051/1051 [==============================] - 0s 126us/step - loss: 0.0018 - mean_squared_error: 0.0018 - val_loss: 0.0913 - val_mean_squared_error: 0.0913
Epoch 149/150
1051/1051 [==============================] - 0s 134us/step - loss: 0.0015 - mean_squared_error: 0.0015 - val_loss: 0.0889 - val_mean_squared_error: 0.0889
Epoch 150/150
1051/1051 [==============================] - 0s 179us/step - loss: 0.0011 - mean_squared_error: 0.0011 - val_loss: 0.0890 - val_mean_squared_error: 0.0890


Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [27]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

1051/1051 [==============================] - 0s 96us/step


[0.0008075568701723657, 0.0008075568701723657]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [28]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

263/263 [==============================] - 0s 104us/step


[0.0890125596166337, 0.0890125596166337]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [29]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

146/146 [==============================] - 0s 142us/step


[0.17922606778471437, 0.17922606778471437]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [30]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

33268.412465658155

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.